# Data TSA
Your own personal on-demand data defence force.

Data TSA is a generic data profiling tool that provides data type-specific inspection tools for your data. Additionally, metrics can be split over any slicer field, providing insight into how metrics change over periods of time, versions of software, etc.

Here's  quick example using some sample data.

In [1]:
import pandas as pd
from data_tsa.sample_data import SampleData
from data_tsa.profiler import Profiler

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

# Generate test data
samp = SampleData(1000)
df = samp.get_sample_data()
df.head()

,id,partial_null,created_at,duplicate_string,string_slicer,mixed_precision_datetime,date_string,mixed_sign_numbers
0,0,NaN,2018-01-01 00:00:00,test,B,2019-01-01 00:00:00,1/1/2019 12:00,-1
1,1,NaN,2018-01-01 06:00:00,Testing,A,2019-01-01 01:01:00,1/1/2019,1
2,2,NaN,2018-01-01 12:00:00,Test,D,2019-01-01 01:01:00,1/1/2019,-1
3,3,NaN,2018-01-01 18:00:00,Test,B,2019-01-01 01:01:00,1/1/2019 12:00:00,-1
4,4,NaN,2018-01-02 00:00:00,Test,D,2019-01-01 00:00:00,1/1/2019 12:00,1


In [2]:
# Initialize Profiler with no slicer
profiler = Profiler(df)

# Profile dataframe columns
p1_output = profiler.profile()
p1_output.head(20)

,inspector,column,slice,measure,measure_value
0,number,id,None,row_count,1000
1,number,id,None,distinct_count,1000
2,number,id,None,null_ratio,0
3,number,id,None,min_value,0
4,number,id,None,max_value,999
5,number,id,None,negative_ratio,0
6,number,id,None,mean_value,499.5
7,number,id,None,median_value,499.5
8,number,id,None,stdev,288.819
9,number,id,None,zero_ratio,0.001


In [3]:
# Initialize Profiler with slcer
profiler_slicer = Profiler(df, slicer='string_slicer')

# Profile dataframe columns over slicer partitions
p2_output = profiler_slicer.profile(lags=3)
p2_output.head(10)

1 / 4
2 / 4
3 / 4
4 / 4


,inspector,column,slice,measure,measure_value,l1_measure_value,l2_measure_value,l3_measure_value
3,datetime,created_at,A,conversion_error_indicator,0,None,None,None
3,datetime,created_at,B,conversion_error_indicator,0,0,None,None
3,datetime,created_at,C,conversion_error_indicator,0,0,0,None
3,datetime,created_at,D,conversion_error_indicator,0,0,0,0
1,datetime,created_at,A,distinct_count,263,None,None,None
1,datetime,created_at,B,distinct_count,248,263,None,None
1,datetime,created_at,C,distinct_count,239,248,263,None
1,datetime,created_at,D,distinct_count,250,239,248,263
5,datetime,created_at,A,max_value,2018-09-07 18:00:00,None,None,None
5,datetime,created_at,B,max_value,2018-09-05 00:00:00,2018-09-07 18:00:00,None,None


In [4]:
# See how metrics change over slices for any given column
profiler_slicer.show_column_result('duplicate_string')

slice  measure_value                                                                                                                        
measure       distinct_count empty_ratio null_ratio redundancy_indicator row_count special_character_ratio strict_distinct_count trim_required_ratio
0           A              5           0          0                    1       263                       0                     2            0.205323
1           B              5           0          0                    1       248                       0                     2            0.205645
2           C              5           0          0                    1       239                       0                     2             0.16318
3           D              5           0          0                    1       250                       0                     2               0.168

## DataFrameInspector
The __DataFrameInspector__ class is not included by default in the Profiler class, but is a part of the data_tsa package.

It's one and only function at this time is to detect perfect duplicates (i.e. rows that are exactly identical).

In [5]:
from data_tsa.dataframe_inspector import DataFrameInspector

df_dupes = pd.DataFrame({'a': [0, 0], 'b': [0, 0]})
df_dupes

,a,b
0,0,0
1,0,0


In [6]:
# Apply DataFrameInspector
insp = DataFrameInspector(df_dupes)
insp.get_duplicate_row_indicator()

True

In [7]:
# DataFrameInspector will also return a dataframe containing only the duplicate rows
insp.get_duplicate_rows()

,a,b,85706ded-fe06-4716-bbc5-be2fb862bbff
0,0,0,b4b147bc522828731f1a016bfa72c073
1,0,0,b4b147bc522828731f1a016bfa72c073


## Anomaly Detection
When the `Profiler.profile()` method is run, it will create a specified number of lagging values for each column / metric value for a given slice (see example below). 

The __AnomalyDetector__ class takes a Profile object that has had the `profile()` function run and uses the lagging values to detect anomalies for a specified slice (the max value slice, by default). Anomalies are detected by checking the nature and magnitude of the target column / metric value against that of the lagging values. The `AnomalyDetector.detect()` method will compute anomalies and return a summary report, which lists columns in descending order by the number of anomalies detected. 

In [8]:
from data_tsa.anomaly_detector import AnomalyDetector

ad = AnomalyDetector(profiler_slicer)
ad.detect()

,column,anomaly_score
1,mixed_sign_numbers,12
0,created_at,8


This shows that two of the columns in our test data have anomalous metrics for the most recent slice (slice 'D').

The `ad.column_summary()` function will return the anomaly detection outcomes for a specific field:

In [9]:
ad.column_summary('mixed_sign_numbers')

,inspector,column,slice,measure,rule,reference_lags,flag,anomaly_score
189,number,mixed_sign_numbers,D,mean_value,get_abs_perc_error_flag,1,1,1
190,number,mixed_sign_numbers,D,mean_value,get_abs_perc_error_flag,2,1,2
191,number,mixed_sign_numbers,D,mean_value,get_abs_perc_error_flag,3,1,3
198,number,mixed_sign_numbers,D,median_value,get_abs_perc_error_flag,1,1,1
199,number,mixed_sign_numbers,D,median_value,get_abs_perc_error_flag,2,1,2
200,number,mixed_sign_numbers,D,median_value,get_abs_perc_error_flag,3,1,3


Column definitions for the above table:
* inspector: the type of inspector used for the column (a function of data type)
* column: a column from the original input data set
* slice: the slice being tested by the AnomalyDetector class
* measure: the Profiler metric on which an anomaly was detected
* rule: the anomaly detection rule that was triggered when comparing the current column/metric value to a span of lagging values
* reference_lags: the number of lagging values used in comparing against the current slice column/metric value
* flag: binary indication of whether or not the an anomaly was detected
* anomaly_score: __reference_lags__ * __flag__; flags derived from rules using more lagging values are stronger indicators of meaningful variance.

Note that the __anomaly_score__ column returned by the `AnomalyDetector.detect()` function sums the __anomaly_score__ values for each column and sorts in descending order.

The AnomalyDetector class has five anomaly checks (rules) that it performs:
* get_zero_ratio_flag - if the lagging column/metric values are all non-zero but the current slice column/metric value are zero then 1, else 0
* get_positive_ratio_flag - if the lagging column/metric values are all zero but the current slice column/metric value are non-zero then 1, else 0
* get_single_value_flag - if the lagging column/metric values are greater than one but the current slice column/metric value equals 1 then 1, else 0
* get_abs_perc_error_flag - if the absolute percentage difference (error) between the average of the lagging column/metric values and the current slice column/metric value is greater than 100% then 1, else 0
* get_consistency_flag - if the maximum value of the current slice column/metric is less than the maximum value of the lagging column/metric values then 1 else 0. (Also checks for consistency in the MIN direction).